In [ ]:
# Load drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.listdir()

['.config', 'gdrive', 'sample_data']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import json
key_file = open('/content/gdrive/MyDrive/MUStARD-master/data/sarcasm_data.json')
data = json.load(key_file)

key_order = list(data.keys())

In [ ]:
# Load Individual Data

import pickle
import numpy as np

text_loc = "/content/gdrive/MyDrive/Pickle_Data/text_bert_embeddings.pkl"
audio_loc = "/content/gdrive/MyDrive/Pickle_Data/audio_features.pkl"
video_loc = "/content/gdrive/MyDrive/Pickle_Data/video_features.pkl"
label_map_loc = "/content/gdrive/MyDrive/Pickle_Data/label_map.pkl"

# Text features are available as numpy array in the key_order order
text_feats = pickle.load(open(text_loc, "rb"))

# Audio and Video Features are available as dict in random order - We pre-process them
audio_dict = pickle.load(open(audio_loc, "rb"))
video_dict = pickle.load(open(video_loc, "rb"))

# Label map: Key ---> Ground Truth Label
label_map = pickle.load(open(label_map_loc, "rb"))

# Final Extracted Features
audio_feats = []
video_feats = []

# Ground Truth Labels
y = []

for key in key_order:
  audio_feats.append(audio_dict[key])
  video_feats.append(video_dict[key])
  y.append(label_map[key])

text_feats = np.array(text_feats).astype('float32')
audio_feats = np.array(audio_feats).astype('float32')
video_feats = np.array(video_feats).astype('float32')
y = np.array(y).astype('float32')

In [ ]:
np.shape(text_feats), np.shape(audio_feats), np.shape(video_feats)

((690, 768), (690, 283), (690, 2048))

# Random Forest

In [ ]:
# Concatenate all the modalities: Text - Audio - Video
X = np.concatenate([text_feats, audio_feats, video_feats], axis=1)

# Scale all the data values to 0-1 using min-max scalar
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
text_feats = scaler.fit_transform(text_feats)
audio_feats = scaler.fit_transform(audio_feats)
video_feats = scaler.fit_transform(video_feats)

In [ ]:
# import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
import seaborn as sns

In [ ]:
#define model
rfc = RandomForestClassifier()
parameters = {
    "n_estimators":[5,10,50,100,250],
    "max_depth":[2,4,8,16,32,None]
    
}

In [ ]:
# use grid search cross validation to get best parameters for rfc
def display_best_params(X,y,model,parameters):
    cv = GridSearchCV(model,parameters,cv=5)
    cv.fit(X,y)
    print(f'Best parameters are: {cv.best_params_}')
    print("\n")
    mean_score = cv.cv_results_['mean_test_score']
    std_score = cv.cv_results_['std_test_score']
    params = cv.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')


## All modalities

In [ ]:
# find best parameters for rfc - all modalities case
display_best_params(X,y)

Best parameters are: {'max_depth': 2, 'n_estimators': 100}


0.62 + or -0.109 for the {'max_depth': 2, 'n_estimators': 5}
0.61 + or -0.102 for the {'max_depth': 2, 'n_estimators': 10}
0.645 + or -0.126 for the {'max_depth': 2, 'n_estimators': 50}
0.648 + or -0.132 for the {'max_depth': 2, 'n_estimators': 100}
0.648 + or -0.119 for the {'max_depth': 2, 'n_estimators': 250}
0.599 + or -0.068 for the {'max_depth': 4, 'n_estimators': 5}
0.62 + or -0.094 for the {'max_depth': 4, 'n_estimators': 10}
0.633 + or -0.125 for the {'max_depth': 4, 'n_estimators': 50}
0.643 + or -0.113 for the {'max_depth': 4, 'n_estimators': 100}
0.636 + or -0.135 for the {'max_depth': 4, 'n_estimators': 250}
0.596 + or -0.064 for the {'max_depth': 8, 'n_estimators': 5}
0.599 + or -0.087 for the {'max_depth': 8, 'n_estimators': 10}
0.63 + or -0.106 for the {'max_depth': 8, 'n_estimators': 50}
0.63 + or -0.12 for the {'max_depth': 8, 'n_estimators': 100}
0.635 + or -0.127 for the {'max_depth': 8, 'n_estimators': 25

In [ ]:
#testing all features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1,stratify=y)
forest=RandomForestClassifier(max_depth=2,n_estimators=100)
forest.fit(X_train, y_train)
y_pred_test = forest.predict(X_test)
print(accuracy_score(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print(print(classification_report(y_test, y_pred_test)))

0.6634615384615384
[[26 26]
 [ 9 43]]
              precision    recall  f1-score   support

         0.0       0.74      0.50      0.60        52
         1.0       0.62      0.83      0.71        52

    accuracy                           0.66       104
   macro avg       0.68      0.66      0.65       104
weighted avg       0.68      0.66      0.65       104

None


## Text modality

In [ ]:
# find best parameters for rfc - text modalities case
display_best_params(text_feats,y)

Best parameters are: {'max_depth': 32, 'n_estimators': 250}


0.575 + or -0.04 for the {'max_depth': 2, 'n_estimators': 5}
0.584 + or -0.054 for the {'max_depth': 2, 'n_estimators': 10}
0.594 + or -0.064 for the {'max_depth': 2, 'n_estimators': 50}
0.629 + or -0.075 for the {'max_depth': 2, 'n_estimators': 100}
0.607 + or -0.064 for the {'max_depth': 2, 'n_estimators': 250}
0.551 + or -0.043 for the {'max_depth': 4, 'n_estimators': 5}
0.59 + or -0.071 for the {'max_depth': 4, 'n_estimators': 10}
0.597 + or -0.06 for the {'max_depth': 4, 'n_estimators': 50}
0.601 + or -0.071 for the {'max_depth': 4, 'n_estimators': 100}
0.617 + or -0.075 for the {'max_depth': 4, 'n_estimators': 250}
0.548 + or -0.033 for the {'max_depth': 8, 'n_estimators': 5}
0.597 + or -0.035 for the {'max_depth': 8, 'n_estimators': 10}
0.609 + or -0.084 for the {'max_depth': 8, 'n_estimators': 50}
0.623 + or -0.073 for the {'max_depth': 8, 'n_estimators': 100}
0.635 + or -0.073 for the {'max_depth': 8, 'n_estimators'

In [ ]:
#testing text features
X_train, X_test, y_train, y_test = train_test_split(text_feats, y, test_size=0.15, random_state=1,stratify=y)
forest=RandomForestClassifier(max_depth=32,n_estimators=250)####
forest.fit(X_train, y_train)
y_pred_test = forest.predict(X_test)
print(accuracy_score(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print(print(classification_report(y_test, y_pred_test)))

0.6826923076923077
[[32 20]
 [13 39]]
              precision    recall  f1-score   support

         0.0       0.71      0.62      0.66        52
         1.0       0.66      0.75      0.70        52

    accuracy                           0.68       104
   macro avg       0.69      0.68      0.68       104
weighted avg       0.69      0.68      0.68       104

None


## Audio modality

In [ ]:
# find best parameters for rfc - all modalities case
display_best_params(audio_feats,y)

Best parameters are: {'max_depth': 16, 'n_estimators': 100}


0.645 + or -0.114 for the {'max_depth': 2, 'n_estimators': 5}
0.63 + or -0.121 for the {'max_depth': 2, 'n_estimators': 10}
0.639 + or -0.12 for the {'max_depth': 2, 'n_estimators': 50}
0.632 + or -0.125 for the {'max_depth': 2, 'n_estimators': 100}
0.645 + or -0.129 for the {'max_depth': 2, 'n_estimators': 250}
0.596 + or -0.07 for the {'max_depth': 4, 'n_estimators': 5}
0.626 + or -0.111 for the {'max_depth': 4, 'n_estimators': 10}
0.645 + or -0.119 for the {'max_depth': 4, 'n_estimators': 50}
0.643 + or -0.128 for the {'max_depth': 4, 'n_estimators': 100}
0.645 + or -0.129 for the {'max_depth': 4, 'n_estimators': 250}
0.614 + or -0.079 for the {'max_depth': 8, 'n_estimators': 5}
0.599 + or -0.096 for the {'max_depth': 8, 'n_estimators': 10}
0.641 + or -0.126 for the {'max_depth': 8, 'n_estimators': 50}
0.643 + or -0.124 for the {'max_depth': 8, 'n_estimators': 100}
0.641 + or -0.129 for the {'max_depth': 8, 'n_estimators'

In [ ]:
#testing audio features
X_train, X_test, y_train, y_test = train_test_split(audio_feats, y, test_size=0.15, random_state=1,stratify=y)
forest=RandomForestClassifier(max_depth=16,n_estimators=100)####
forest.fit(X_train, y_train)
y_pred_test = forest.predict(X_test)
print(accuracy_score(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print(print(classification_report(y_test, y_pred_test)))

0.6442307692307693
[[27 25]
 [12 40]]
              precision    recall  f1-score   support

         0.0       0.69      0.52      0.59        52
         1.0       0.62      0.77      0.68        52

    accuracy                           0.64       104
   macro avg       0.65      0.64      0.64       104
weighted avg       0.65      0.64      0.64       104

None


## Video modality

In [ ]:
# find best parameters for rfc - all modalities case
display_best_params(video_feats,y)

Best parameters are: {'max_depth': 4, 'n_estimators': 100}


0.575 + or -0.034 for the {'max_depth': 2, 'n_estimators': 5}
0.571 + or -0.067 for the {'max_depth': 2, 'n_estimators': 10}
0.601 + or -0.071 for the {'max_depth': 2, 'n_estimators': 50}
0.599 + or -0.115 for the {'max_depth': 2, 'n_estimators': 100}
0.604 + or -0.101 for the {'max_depth': 2, 'n_estimators': 250}
0.559 + or -0.073 for the {'max_depth': 4, 'n_estimators': 5}
0.597 + or -0.081 for the {'max_depth': 4, 'n_estimators': 10}
0.596 + or -0.085 for the {'max_depth': 4, 'n_estimators': 50}
0.639 + or -0.104 for the {'max_depth': 4, 'n_estimators': 100}
0.62 + or -0.118 for the {'max_depth': 4, 'n_estimators': 250}
0.561 + or -0.05 for the {'max_depth': 8, 'n_estimators': 5}
0.542 + or -0.067 for the {'max_depth': 8, 'n_estimators': 10}
0.601 + or -0.094 for the {'max_depth': 8, 'n_estimators': 50}
0.616 + or -0.098 for the {'max_depth': 8, 'n_estimators': 100}
0.613 + or -0.119 for the {'max_depth': 8, 'n_estimators'

In [ ]:
#testing video features
X_train, X_test, y_train, y_test = train_test_split(video_feats, y, test_size=0.15, random_state=1,stratify=y)
forest=RandomForestClassifier(max_depth=4,n_estimators=100)####
forest.fit(X_train, y_train)
y_pred_test = forest.predict(X_test)
print(accuracy_score(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print(print(classification_report(y_test, y_pred_test)))

0.6153846153846154
[[33 19]
 [21 31]]
              precision    recall  f1-score   support

         0.0       0.61      0.63      0.62        52
         1.0       0.62      0.60      0.61        52

    accuracy                           0.62       104
   macro avg       0.62      0.62      0.62       104
weighted avg       0.62      0.62      0.62       104

None


# Bagging Classifier

In [ ]:
bc = BaggingClassifier()

parameters = {
    "n_estimators":[5,10,50,100,250],    
}

## All modalities

In [ ]:
display_best_params(X,y,bc,parameters)

KeyboardInterrupt: ignored

In [ ]:
#testing all features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1,stratify=y)
forest=BaggingClassifier(n_estimators=100)#####
forest.fit(X_train, y_train)
y_pred_test = forest.predict(X_test)
print(accuracy_score(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print(print(classification_report(y_test, y_pred_test)))

0.6826923076923077
[[30 22]
 [11 41]]
              precision    recall  f1-score   support

         0.0       0.73      0.58      0.65        52
         1.0       0.65      0.79      0.71        52

    accuracy                           0.68       104
   macro avg       0.69      0.68      0.68       104
weighted avg       0.69      0.68      0.68       104

None


## Text modality

In [ ]:
# find best parameters for bc - text modalities case
display_best_params(text_feats,y)

In [ ]:
#testing text features
X_train, X_test, y_train, y_test = train_test_split(text_feats, y, test_size=0.15, random_state=1,stratify=y)
forest=BaggingClassifier(n_estimators=100)####
forest.fit(X_train, y_train)
y_pred_test = forest.predict(X_test)
print(accuracy_score(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print(print(classification_report(y_test, y_pred_test)))

0.5865384615384616
[[29 23]
 [20 32]]
              precision    recall  f1-score   support

         0.0       0.59      0.56      0.57        52
         1.0       0.58      0.62      0.60        52

    accuracy                           0.59       104
   macro avg       0.59      0.59      0.59       104
weighted avg       0.59      0.59      0.59       104

None


## Audio Modality

In [ ]:
# find best parameters for bc - audio modality case
display_best_params(audio_feats,y)

In [ ]:
#testing audio features
X_train, X_test, y_train, y_test = train_test_split(audio_feats, y, test_size=0.15, random_state=1,stratify=y)
forest=BaggingClassifier(n_estimators=100)####
forest.fit(X_train, y_train)
y_pred_test = forest.predict(X_test)
print(accuracy_score(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print(print(classification_report(y_test, y_pred_test)))

0.6442307692307693
[[29 23]
 [14 38]]
              precision    recall  f1-score   support

         0.0       0.67      0.56      0.61        52
         1.0       0.62      0.73      0.67        52

    accuracy                           0.64       104
   macro avg       0.65      0.64      0.64       104
weighted avg       0.65      0.64      0.64       104

None


## Video Modality

In [ ]:
# find best parameters for bc - video modality case
display_best_params(video_feats,y)

In [ ]:
#testing video features
X_train, X_test, y_train, y_test = train_test_split(video_feats, y, test_size=0.15, random_state=1,stratify=y)
forest=BaggingClassifier(n_estimators=100)####
forest.fit(X_train, y_train)
y_pred_test = forest.predict(X_test)
print(accuracy_score(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print(print(classification_report(y_test, y_pred_test)))

0.6442307692307693
[[33 19]
 [18 34]]
              precision    recall  f1-score   support

         0.0       0.65      0.63      0.64        52
         1.0       0.64      0.65      0.65        52

    accuracy                           0.64       104
   macro avg       0.64      0.64      0.64       104
weighted avg       0.64      0.64      0.64       104

None


# XG Boost Classifier

In [ ]:
bc = BaggingClassifier()

parameters = {
    "n_estimators":[5,10,50,100,250],    
}